In [1]:
# load libraries

import pyarrow.parquet as pq
import os

import pandas as pd
import numpy as np

import re

import matplotlib.pyplot as plt
%matplotlib inline

from functions import utils, preprocessing, headlines
from mab import mab_filter

## Load data

In [2]:
dirpath = '.\data' # this will be used eventually-- need to un-pgp the files first (can't remember how!!!)

dirpath = './data/sample_202111111'

dirpath + '/product.parquet/'
product_df = utils.get_pq_df(dirpath + '/product.parquet/')
store_df = utils.get_pq_df(dirpath + '/store.parquet/')
ar_df = utils.get_pq_df(dirpath + '/action_reward.parquet/')
weather_df = utils.get_pq_df(dirpath + '/weather.parquet/')

In [23]:
#preprocessing- product df
# get form codes from product names
# get notional flavor from product names
ice_keywords = ['refreshers', 'frappuccino', 'blended', 'cold', 'iced', 'bottled']
product_df2 = preprocessing.get_form_codes(product_df, ice_keywords)
prodcut_df2= preprocessing.get_notional_flavor(product_df2)

# preprocessing- store df
# get city state from zip codes
store_df2 = preprocessing.get_zipcodes_from_csv(store_df)

## Given data

In [24]:
# example- the 4 recomended products are given

products_given = ['chai-tea', 'toasted-white-hot-chocolate', 'clover-x-costa-rica-naranjo-hot']

In [25]:
iced_products = ['apple-crisp-frappuccino-blended-beverage',
                 'iced-reserve-bar-caffè-americano',
                 'iced-espresso-classics---skinny-vanilla-latte',
                'toasted-white-chocolate-mocha-frappuccino']

In [26]:
hour1 = 8
hour2 = 15
hour3 = 18

store1 = 2085 # I think this is the reserve on pike edit: don't use this, no weather data
store2 = 53313 # this should be the reserve in chicago on michigan ave
store3 = 9447 # this is the one on 15th around interbay/magnolia bridge which actually has a drivethru
store4 = 9810 # some drivethru store in arkanas

In [27]:
hg = headlines.HeadlineGenerator(store_df2, product_df2, weather_df)

In [28]:
hg.get_headlines(store2, hour1, products_given)

['Sunny in Chicago',
 'Morning in Chicago',
 'Sunny Morning in Chicago',
 'Morning Light Pick Me Up',
 'Sunny Light Pick Me Up',
 'Morning Treat',
 'Sunny Treat',
 'Morning Boost',
 'Sunny Boost',
 'Morning Flavor',
 'Sunny Flavor']

In [29]:
interbay_headlines = hg.get_headlines(store3, hour3, products_given)

In [30]:
interbay_headlines

['Sunny in Seattle',
 'Evening in Seattle',
 'Sunny Evening in Seattle',
 'Evening Light Pick Me Up',
 'Sunny Light Pick Me Up',
 'Evening Treat',
 'Sunny Treat',
 'Evening Boost',
 'Sunny Boost',
 'Evening Flavor',
 'Sunny Flavor']

In [31]:
hot_weather_headlines = hg.get_headlines(store4, hour1, iced_products)

In [32]:
hot_weather_headlines

['Sunny in Conway',
 'Morning in Conway',
 'Sunny Morning in Conway',
 'Morning Light Pick Me Up',
 'Sunny Light Pick Me Up',
 'Morning Treat',
 'Sunny Treat',
 'Morning Boost',
 'Sunny Boost',
 'Morning Flavor',
 'Sunny Flavor']

## Simulation

In [33]:
# parameters: headlines, num_turns, lbound=0.01, ubound=0.15

mab = mab_filter.MultiArmBandits(interbay_headlines, num_turns=1000)

In [34]:
mab.simulation(verbose=True)

Headline 0 was chosen 95.0 times
Headline 1 was chosen 63.0 times
Headline 2 was chosen 49.0 times
Headline 3 was chosen 46.0 times
Headline 4 was chosen 238.0 times
Headline 5 was chosen 53.0 times
Headline 6 was chosen 98.0 times
Headline 7 was chosen 58.0 times
Headline 8 was chosen 126.0 times
Headline 9 was chosen 82.0 times
Headline 10 was chosen 92.0 times

Overall conclusion: best headline is 4

Sunny Light Pick Me Up


In [35]:
mab_iced = mab_filter.MultiArmBandits(hot_weather_headlines, num_turns=1000)

In [36]:
mab_iced.simulation(verbose=True)

Headline 0 was chosen 28.0 times
Headline 1 was chosen 29.0 times
Headline 2 was chosen 73.0 times
Headline 3 was chosen 40.0 times
Headline 4 was chosen 65.0 times
Headline 5 was chosen 98.0 times
Headline 6 was chosen 28.0 times
Headline 7 was chosen 42.0 times
Headline 8 was chosen 45.0 times
Headline 9 was chosen 522.0 times
Headline 10 was chosen 30.0 times

Overall conclusion: best headline is 9

Morning Flavor
